.

<span style="color: #8B008B; font-family: Arial; font-size: 3em;">Sistema de recomendación de librerías Python</span>

 <span style="color:#4B0082; font-family: Arial; font-size: 2em;">José Pérez Yázquez</span>

 <span style="color:#4B0082; font-family: Arial; font-size: 1.5em;">Octubre de 2017</span>

---

<span style="color: #12297c; font-family: Arial; font-size: 3em;">Introducción</span>

El presente proyecto implementa una propuesta de recomendador de librerías Python, la idea es que dado un proyecto que estemos desarrollando, nos recomiende librerías que otros proyectos similares han usado. Para ello usa como "corpus" la BBDD por excelencia de código, **Github**, y como medida de similitud sus descripciones y las librerías que ya compartan.

El proyecto consta de dos fases, en la primera de ellas obtendremos todos los metadatos de los proyectos usando la API Rest de Github y en la segunda parte, basándonos en esos metadatos construiremos el sistema de recomendación.

Aunque el objetivo del proyecto es procesar proyectos **Python**, sería fácilmente extensible para que procesara proyectos implementados en otros lenguajes. Por ejemplo, si quisiéramos hacer un sistema de recomendación de librerías **R**, solo sería necesario modificar la expresión regular que busca los **import** para que buscase la instrucción **library**.



        

.



<span style="color: #12297c; font-family: Arial; font-size: 3em;">Carga y procesamiento de datos</span>        

Esta fase se divide a su vez en cuatro subfases:
 - Recopilación de la información de los proyectos
 - Clonado de los proyectos
 - Procesamiento de los proyectos clonados
 - Preprocesamiento de los datos

## Recopilación de la información de los proyectos

Usando la API Rest de Github descarga la información, metadatos, de todos los proyectos que cumplen una serie de criterios, estos serán los siguientes:
  - Identificados como proyectos Python
  - Creado entre el 1 de Enero de 2012 y el 31 de Agosto de 2017
  - Con un mínimo de 10 estrellas

Toda la información es almacenada en MongoDB concretamente en una colección llamada **projects** de una BD llamada **github**, la información almacenada es la siguiente:
 - **id**: Identificador generado por Github
 - **name**: Nombre del proyecto
 - **full_name**: Nombre ampliado del proyecto
 - **created_at**: Fecha de creación
 - **git_url**: Url al repositorio (es lo que se usa cuando se quiere clonar)
 - **description**: Descripción del proyecto
 - **language**: Lenguaje de programación del proyecto (Java, Python, Scala, etc)
 - **stars**: Número de estrellas del proyecto, nos da una medida de la *populariodad* del proyecto
 - **done**: Se usa a nivel interno, indica si un proyecto está procesado, esto es identificadas las librerías que usa y el fichero README cargado, en los puntos siguientes se detallan estos dos datos
 - **readme_txt**: Almacenará el texto de los ficheros README de los proyectos, será la información base para el sistema de recomendación
 - **library**: Lista con las librerías que el proyecto usa, en principio estará vacía y se alimentará cuando se procese el proyecto
 - **raw_data**: Almacena toda la información del proyecto sin procesar por si más adelante es necesario algún dato que en estos momentos no parece relevante

Cuando se usa la API de Github sin autenticación existen una serie de límites que no podemos superar, el primero es que el número máximo de resultados que cualquier consulta puede devolver
es de 1000, para salvar este impedimento, vamos a realizar sucesivas consultas restringiendo las llamadas a un solo día del intervalo que vamos a cubrir.

Por otra parte, existe otra restricción en el uso de la API, el número de llamadas por minuto está limitado a 60, podríamos incrementar esta cantidad si las llamadas son autenticadas,
esto es usar por ejemplo un "client ID" y "secret"  como partes de la consulta, pero vamos abordar una estrategia diferente. Se irán realizando las consultas sin atender a los límites y cuando ese límite se supere, se suspenderá el proceso durante 61 segundos y posteriormente se reanudará
por el punto en el que iba.

Si se desea consultar los límites comentados, podemos hacer una consulta, por ejemplo usando curl

curl -i https://api.github.com/users/octocat

HTTP/1.1 200 OK

Date: Mon, 01 Jul 2017 17:27:06 GMT

Status: 200 OK

X-RateLimit-Limit: 60

X-RateLimit-Remaining: 56

X-RateLimit-Reset: 1372700873


En la cabecera de la respuesta nos muestra los límites con el valor actual:

 - **X-RateLimit-Limit**       Número máximo de solicitudes que se le permite hacer por hora.
 - **X-RateLimit-Remaining**   Número de solicitudes restantes en la ventana de límite de velocidad actual.
 - **X-RateLimit-Reset**       Hora a la que se restablece la ventana de límite de velocidad actual en segundos UTC.

En primer lugar vamos a cargar las librerías necesarias para esta fase del proyecto

In [1]:
%matplotlib inline
import warnings
import logging
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import requests
import json
import os
import sys
import re
from time import sleep
from pymongo import MongoClient
from datetime import date, timedelta

A continuación vamos a definir una serie de constantes que serán usadas en el código, de esta manera podremos cambiar facilimente alguno de estos valores.
El significado de cada una de ellas es el siguiente:
 - **MIN_STARTS**: Número mínimo de estrellas que tiene que tener un proyecto para ser considerado
 - **START_DATE**: Fecha de inicio del intervalo, haciendo referencia a la fecha de creación del proyecto en Github
 - **END_DATE**: Fecha de fin del intervalo
 - **URL_PATTERN**: Patrón de la URL para la consulta a la API de Github, podemos ver que filtramos por lenguaje (Python), por fecha de creación (la suministraremos durante la ejecución) y por numero de estrellas (referido a la constante comentada anteriormente).
 - **ROOT_PATH**: Directorio raiz donde clonaremos los repositorios (debe existir antes de empezar a clonar los proyectos).
 - **CLONE_COMMAND**: Comando usado para clonar los repositorios
 - **LIBRARY_PATTERN**: Patrón que cumplen las líneas de código en las que se importa una librería
 - **FILE_LANGUAGE_EXTENSION**: Extensión de los ficheros para el lenguaje a procesar.

En la introducción deciamos que el sistema podrías ser extensible a otros lenguajes más allá de Python, para ello tendríamos que cambiar las dos últimas constantes a las apropiadas al lenguaje que queramos procesar, por ejemplo para **Java**, la última constante valdría **".java"** y para **R** valdría **".r"**

In [3]:
MIN_STARTS = 10
START_DATE = date(2017, 8, 1)
END_DATE = date(2017, 8, 2)
URL_PATTERN = 'https://api.github.com/search/repositories?q=language:Python+created:{0}+stars:>={1}&type=Repositories'

CLONE_COMMAND = "git clone {0} {1}"
ROOT_PATH = "d:/tfm/tmp"

LIBRARY_PATTERN = '(?m)^(?:from[ ]+(\S+)[ ]+)?import[ ]+(\S+)(?:[ ]+as[ ]+\S+)?[ ]*$'
FILE_LANGUAGE_EXTENSION = ".py"

Como se ha comentado anteriormente, vamos a usar MongoDB como tecnología de persistencia, vamos a definir una funcion que nos devuelva una referencia a la colección de proyectos con la que estamos trabajando.

In [4]:
def get_repository():
    client = MongoClient('localhost', 27017)
    db = client.github_poc
    return db.projects

La siguiente función es la que realmente hace las llamadas a la API, tendrá como entrada la fecha de creación de los proyectos, usará la URL que hemos definicdo como constante.

In [5]:
def get_projects_by_date(date):
    print("Processing date", date)
    url_pattern = URL_PATTERN
    url = url_pattern.format(date, MIN_STARTS)
    response = requests.get(url)
    if (response.ok):
        response_data = json.loads(response.content.decode('utf-8'))['items']
        for project in response_data:
            insert_project(project)
    else:
        response.raise_for_status()

A continuanción definimos otra función auxiliar, que dada la información de la respuesta de la API, seleccionamos los atributos que vamos a necesitar y los almacenamos como un documento Mongo. Adicionalmente crea las propiedades mencionadas previamente (readme_txt,library,etc.).

In [6]:
def insert_project(github_project):
    if repository_projects.find_one({"id": github_project["id"]}):
        print("Project {0} is already included in the repository".format(github_project["name"]))
    else:
        project = {
            'id': github_project["id"],
            'name': github_project["name"],
            'full_name': github_project["full_name"],
            'created_at': github_project["created_at"],
            'git_url': github_project["git_url"],
            'description': github_project["description"],
            'language': github_project["language"],
            'stars': github_project["stargazers_count"],
            'done': False,
            'readme_txt': "",
            'library': [],
            'raw_data': github_project
        }
        repository_projects.insert(project)

Finalmente ejecutamos el programa que, usando las funciones anteriormente definidas, descarga la información de los proyectos y la inserta en la BBDD, podemos observar que descompone las llamadas para traer en cada una de ellas solo los proyectos de un determinado dia y que *"gestiona"* las restricciones que tenemos en cuanto a llamada en la ventana de tiempo actual. 

En primer lugar recuperamos la colección en la que vamos a insertar los proyectos.

In [7]:
repository_projects = get_repository()

Cargamos los proyectos en la colección, aplicamos las técnicas descritas para salvar las restricciones que nos impone la API de Github (número de items devuelto por cada consulta y número de llamadas por minuto).

In [8]:
for project_create_at in [START_DATE + timedelta(days=x) for x in range((END_DATE - START_DATE).days + 1)]:
    try:
        get_projects_by_date(project_create_at)
    except:
        print(">> Reached call limit, waiting 61 seconds...")
        sleep(61)
        get_projects_by_date(project_create_at)

Processing date 2017-08-01
Processing date 2017-08-02


En este punto tendriamos cargada toda la información que necesitamos de Github.

 ## Clonado de los proyectos

Para analizar las librerías que un proyecto utiliza, un camino podría ser buscar los ficheros **requirements** de cada proyecto y parsear esa información, pero dicho fichero no está presente en una gran cantidad de proyectos, por lo que se ha optado por un camino un poco más **radical**.

Usando la información cargada en el paso anterior, fundamentalmente la propiedad **git_url**, vamos a clonar cada proyecto en local para posteriormente procesarlos.

**NOTAS**: 
* El directorio ROOT_PATH debe existir antes de lanzar el siguiente script
* Filtramos por los proyectos que no están procesados y nos aseguramos que el directorio no existe, porque este código, dado que puede tardar bastante, lo vamos a ejecutar en diferentes momentos, es decir llegado un punto se podría abortar el proceso y lanzarlo más adelante sin que se resienta por ello.

In [9]:
os.chdir(ROOT_PATH)

for project in repository_projects.find({'done':False}):
    print("Cloning project", project['name'], "...")
    path = ROOT_PATH + "/" + str(project["id"])
    if not os.path.isdir(path):
        os.system(CLONE_COMMAND.format(project["git_url"], project["id"]))

Cloning project defcon25-public ...
Cloning project isf ...
Cloning project MachineLearningAction ...
Cloning project Deep-Image-Matting ...
Cloning project visimportance ...
Cloning project TWindbg ...
Cloning project vulcan ...
Cloning project WAF_Bypass_Helper ...
Cloning project programmable-agents_tensorflow ...
Cloning project django_rest_example ...
Cloning project keras-transform ...
Cloning project SkySpyWatch ...
Cloning project QQ_zone ...
Cloning project pypaperbak ...
Cloning project Imports-in-Python ...
Cloning project OSINT-SPY ...
Cloning project kinetics-i3d ...
Cloning project cifar-10-cnn ...
Cloning project solving-minesweeper-by-tensorflow ...
Cloning project webcrawler ...
Cloning project parseNTFS ...
Cloning project neural_factorization_machine ...
Cloning project attentional_factorization_machine ...
Cloning project ChatGirl ...
Cloning project pytorch-fitmodule ...
Cloning project Semantic_Segmentation ...
Cloning project captcha-svm ...
Cloning project trafa

En este punto tendríamos clonados todos los repositorios con los que construiremos el sistema de recomendación.

**NOTA**: Son muchos, muchos gigabytes.

## Procesamiento de los proyectos clonados

Comenzamos esta etapa definiendo una función que dado un fichero python (extensión **.py**), lo recorre linea por linea y, usando expresiones regulares, buscamos todas las librerias que se estén usando.
Almacenamos esta lista en la propiedad **library** del proyecto en cuestión.

In [3]:
def process_python_file(project, file_path):
    def add_to_list(item):
        if not item in library:
            library.append(item)

    library = project['library']   
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            match = re.search(LIBRARY_PATTERN, line)
            if match:
                if match.group(1) != None:
                    add_to_list(match.group(1))
                else:
                    add_to_list(match.group(2))
    project['library'] = library
    repository_projects.update({'_id': project['_id']}, {"$set": project}, upsert=False)

Por otra parte, definimos también una función que dado un fichero **README** lo lea y almacena en la propiedad **readme_txt**

In [4]:
def process_readme_file(project, file_path):
    with open(file_path, 'r') as f:
        project['readme_txt'] = f.read()
        repository_projects.update({'_id': project['_id']}, {"$set": project}, upsert=False)

Finalmente recorremos los repositorios no procesados, como comentabamos anteriormente este proceso permite lanzarlo reiteradas veces, para cada repositorio analizamos su fichero **README** y cada uno de los ficheros Python para extraer las librerías.

In [12]:
for project in repository_projects.find({'done':False}):
    try:
        path = ROOT_PATH.format(project["id"])
        if os.path.isdir(path):
            print("Processing project ", project["name"])
            project['done'] = True
            repository_projects.update({'_id': project['_id']}, {"$set": project}, upsert=False)
            for root, dirs, files in os.walk(path):
                for file in files:
                    try:
                        if file.endswith(FILE_LANGUAGE_EXTENSION):
                            process_python_file(project, os.path.join(root, file))
                        else:
                            if file.lower().startswith("readme."):
                                process_readme_file(project, os.path.join(root, file))
                    except:
                        pass
    except:
        print("Error procesing project {0} [{1}] - {2}".format(project['id'], project['name'], sys.exc_info()[0]))

Processing project  defcon25-public
Processing project  isf
Processing project  MachineLearningAction
Processing project  Deep-Image-Matting
Processing project  visimportance
Processing project  TWindbg
Processing project  vulcan
Processing project  WAF_Bypass_Helper
Processing project  programmable-agents_tensorflow
Processing project  django_rest_example
Processing project  keras-transform
Processing project  SkySpyWatch
Processing project  QQ_zone
Processing project  pypaperbak
Processing project  Imports-in-Python
Processing project  OSINT-SPY
Processing project  kinetics-i3d
Processing project  cifar-10-cnn
Processing project  solving-minesweeper-by-tensorflow
Processing project  webcrawler
Processing project  parseNTFS
Processing project  neural_factorization_machine
Processing project  attentional_factorization_machine
Processing project  ChatGirl
Processing project  pytorch-fitmodule
Processing project  Semantic_Segmentation
Processing project  captcha-svm
Processing project  t

# Procesamiento de los datos

Tras la fase anterior, tendríamos una colección MongoDB con cada uno de los proyectos de nuestro "pre-corpus" con la lista de librerías que cada proyecto usa, así como su descripción **"extendida"** extraída de su fichero **README**. Estaríamos en disposición por tanto de comenzar la implementación de nuestro recomendador.

Vamos a implementar dos versiones del recomendador, una que buscará similitudes basándose en las descripciones de los proyectos y otra que se basará en las librerías que cada proyecto usa para determinar proyectos similares.
Para la primera de las opciones se usará un modelo **LSA** (Latent Semantic Analysis) usando procesamiento de lenguaje natural, para la segunda se usará un simple **"Count Vectorizer"**, dado que la aproximación anterior no es adecuada para documentos con pocos términos, en nuestro caso un documento sería el listado de las librerías usadas.


## Carga de los datos


Comenzamos importando las librerías que vamos a usar en esta fase del proyecto.

In [2]:
import os
import uuid
import re
import nltk
import json
import itertools
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
from collections import defaultdict
from collections import OrderedDict
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim import corpora, models, similarities, matutils
from gensim.models import CoherenceModel

Definimos una funcion para cargar los datos de los proyectos, entre los datos a cargar está el fichero **readme** y las librerías usadas, datos que serán los que usemos para calcular las similitudes entre los proyectos.

In [14]:
def load_projects(max_projects=50000):
    def get_repository():
        client = MongoClient('localhost', 27017)
        db = client.github
        return db.projects
    projects_repository = get_repository()
    return list(projects_repository.find({'done': True}).limit(max_projects))

Usando la funcíon anterior cargamos los datos, dado que el proceso es bastante pesado solo cargaremos un subconjunto de las mismas

In [15]:
projects = load_projects(100)

A modo de ejemplo, mostramos las primeras 5 librerías de uno de los proyectos

In [16]:
projects[0]['library'][:5]

['sys', 'os', 'json', 'codecs', 'shutil']

Y una parte del contenido del fichero README

In [78]:
projects[0]['readme_txt'][:5000]

"Glue\n====\n\n.. image:: https://badge.fury.io/py/glue.png\n    :target: http://badge.fury.io/py/glue\n\n.. image:: https://travis-ci.org/jorgebastida/glue.png?branch=master\n    :target: https://travis-ci.org/jorgebastida/glue\n\n.. image:: https://coveralls.io/repos/jorgebastida/glue/badge.png?branch=master\n    :target: https://coveralls.io/r/jorgebastida/glue?branch=master\n\n.. image:: https://pypip.in/d/glue/badge.png\n    :target: https://crate.io/packages/glue/\n\n\nGlue is a simple command line tool to generate sprites::\n\n    $ glue source output\n\n* The latest documentation is available at: http://glue.readthedocs.org\n* Installation instructions: http://glue.readthedocs.org/en/latest/installation.html\n* Glue-users mailing list: https://groups.google.com/forum/#!forum/glue-users\n\nFeatures\n--------\n* Automatic Sprite (Image + Metadata) creation including:\n\n  - css (less, scss)\n  - cocos2d\n  - json (array, hash)\n  - CAAT\n\n* Automatic multi-dpi `retina <http://gl

## Preprocesamiento de los datos

### Descripciones
 
En primer lugar, procesamos las descripciones, quedándonos con las palabras diferentes que encontramos en cada una de ellas, eliminamos puntuaciones, números y palabras irrelevantes como nombres personales y palabras comunes que no aportan significado (denominados "stop words"). Esto evitará que se formen tópicos en torno a ellos

Además, utilizamos el stemmer Snowball, también llamado Porter2 stemmer, para detectar palabras similares presentes en diferentes formatos (eliminar sufijo, prefijo, etc.). Snowball es un lenguaje desarrollado por M.F. Porter, para definir de forma eficiente stemmers. Este algoritmo de derivación es el más utilizado en el dominio del procesamiento del lenguaje natural.

Para hacer el procesamiento usaremos la librería nltk (Natural Language Toolkit), proporciona un gran número de métodos que cubren diferentes temas en el dominio de los datos del lenguaje humano, como la clasificación, derivación, etiquetado, análisis y razonamiento semántico.




In [18]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

Creamos una función que dado un texto lo descompone en las palabras con significado que lo componen. Vamos a excluir los siguientes tipos de palabras:

- Nombres propios
- Palabras consideradas como "Stop Words"
- Números

Adicionalmente aplicamos el proceso de "steamming" que comentabamos anteriormente

In [19]:
def get_words(text):
    def add_word(word):
        word = word.lower()
        if word not in stop_words and not word.replace('.', '').isdigit():
            words.append(stemmer.stem(word))

    words = []
    for chunk in nltk.ne_chunk(nltk.pos_tag(tokenizer.tokenize(text))):
        # nltk.word_tokenize    devuelve la lista de palabras que forman la frase (tokenización)
        # nltk.pos_tag          devuelve el part of speech (categoría) correspondiente a la palabra introducida
        # nltk.ne_chunk         devuelve la etiqueta correspondiente al part of speech (POC)
        try:
            if chunk.label() == 'PERSON':
                # PERSON es un POC asociado a los nombres propios, los cuales no vamos a añadir
                pass
            else:
                for c in chunk.leaves():
                    add_word(c[0])
        except AttributeError:
            add_word(chunk[0])

    return words

Creamos una función que aplica la función anterior a los resúmenes de todos los proyectos.

In [20]:
def get_texts_from_readme(projects):
    texts = []
    [texts.append(get_words(project['readme_txt'])) for project in projects]
    return texts

Finalmente ejecutamos la función que acabamos de definir, tendremos una lista de listas, en la que para cada proyecto tendremos las palabras que lo definen.

In [21]:
texts_from_readme = get_texts_from_readme(projects)

A modo de ejemplo, mostramos las 5 primeras entradas de uno de los proyectos.

In [22]:
texts_from_readme[0][:5]

['glue', 'imag', 'https', 'badg', 'furi']

### Librerías
 
Las librerías también las vamos a "limpiar", este proceso lo podríamos haber hecho cuando cargamos los datos desde los proyectos clonados, pero he preferido hacerlo en un paso adicional para dejar ese proceso lo más simple posible. Por otro lado es posible que con más conocimiento del dominio, pudisen surgir otras posibles mejoras de cara a mejorar la calidad de los datos. 

En principio solo vamos a excluir las librerías cuyo nombre empieza por ".", esto es un atajo que le dice al interprete de Python que busque en el paquete actual antes del resto de paths del PYTHONPATH. 

In [23]:
for project in projects:
    project['library'] = [library for library in project['library'] if not library.startswith('.')]

.

<span style="color: #12297c; font-family: Arial; font-size: 3em;">Implementación de modelos</span>

# Modelo LSA: Latent Semantic Analysis

## Principios teóricos

Para identificar la similitud entre los proyectos basándonos en su descripción, entendiéndose como tal su fichero README, utilizamos el "análisis semántico latente" (LSA, usando la abreviatura en inglés), que es una técnica ampliamente utilizada en el procesamiento del lenguaje natural. LSA transforma cada texto en un vector, en un espacio de características. En nuestro caso, las características son palabras que ocurren en las descripciones. A continuación, se crea una matriz que contiene todos los vectores: las columnas representan las descripciones de los proyectos y las filas representan palabras únicas. Por consiguiente, el número de filas puede ascender a decenas de miles de palabras. 

Con el fin de identificar las características relevantes de esta matriz, usaremos la "descomposición de valores singulares" (SVD, usando la abreviatura en inglés), que es una técnica de reducción de dimensión, se utiliza para reducir el número de líneas -palabras-, manteniendo y resaltando la similitud entre columnas-descripción -. La dimensión de esta matriz de aproximación se establece mediante un hiperparámetro que es el número de temas, comúnmente llamado como tópicos. En este marco, un tópico consiste en un conjunto de palabras con pesos asociados que definen la contribución de cada palabra a la dirección de este tópico. Basándose en esta matriz de aproximación de baja dimensión, la similitud entre dos columnas -descripciones- se calcula utilizando el coseno del ángulo entre estos dos vectores.


## Creación del diccionario
 
El diccionario está formado por la concatenación de todas las palabras que aparecen en algún resumen de alguno de los proyectos. Básicamente esta función mapea cada palabra única con su identificador. Es decir, si tenemos N palabras, lo que conseguiremos al final es que cada proyecto sea representada mediante un vector en un espacio de N dimensiones.
 
Para ello, partiendo de la lista creada en el paso anterior, usaremos la función **corpora** del paquete **gensim**.

El diccionario consiste en una concatenación de palabras únicas de todas las descripciones. Gensim es una biblioteca eficiente para analizar la similitud semántica latente entre documentos.
Este módulo implementa el concepto de Diccionario - un mapeo entre palabras y
sus entes ids.

Los diccionarios pueden ser creados a partir de un corpus y luego pueden ver las frecuencia del documento (eliminación de palabras comunes mediante el método func: `Dictionary.filter_extremes`), guardado / cargado desde el disco (vía: func: `Dictionary.save` y: func:` Dictionary.load`), fusionado con otro diccionario (: func: `Dictionary.merge_with`) etc.

In [24]:
dictionary = corpora.Dictionary(texts_from_readme)
dictionary

Podemos ver la longitud del diccionario creado

In [28]:
print(dictionary)

Dictionary(3050 unique tokens: ['student', 'pyside2', 'charg', 'sometim', 'outlin']...)


La función **token2i** asigna palabras únicas con sus ids. En nuestro caso, la longitud del diccionario es igual a *N* palabras lo que significa que cada descripción del proyecto será representada a través de un espacio vectorial de *N* dimensiones

Mostramos las primeras 10 entradas

In [29]:
list(itertools.islice(dictionary.token2id.items(), 0, 10))

[('student', 723),
 ('pyside2', 829),
 ('charg', 992),
 ('sometim', 1539),
 ('outlin', 2598),
 ('gmail_password', 1327),
 ('improv', 303),
 ('may', 377),
 ('aka', 1280),
 ('v1', 901)]

## Creación del Corpus

Crearemos un corpus con la colección de todos los resúmenes previamente pre-procesados y transformados usando el diccionario. Vamos a convertir los textos a un formato que gensim puede utilizar, esto es, una representación como bolsa de palabras (BoW). Gensim espera ser alimentado con una estructura de datos de corpus, básicamente una lista de "sparce vectors", estos constan de pares (id, score), donde el id es un ID numérico que se asigna al término a través de un diccionario. 

Mostramos el numero de elementos únicos que existen en los documentos (los que componen el diccionario) y el numero de textos (correspondientes a las descripciones de los proyectos).

In [30]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(texts_from_readme))

Number of unique tokens: 3050
Number of documents: 100


In [31]:
def create_corpus(texts):
    return [dictionary.doc2bow(text) for text in texts]

In [32]:
corpus = create_corpus(texts_from_readme)

A modo de ejemplo, mostramos las 5 primeras entradas del primer proyecto

In [33]:
corpus[0][:5]

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1)]

## Creación del modelo TFID

Un alto peso en tf-idf se alcanza por una alta frecuencia en un Documento y una baja frecuencia en toda la colección de documentos; los pesos tienden a filtrar términos comunes. Para la creación de este corpus, vamos a usar la función **TfidfModel** del objeto **models** (perteneciente a la librería *gemsim*).


In [34]:
def create_tfidf(corpus):
    print("Creación del Modelo Espacio-Vector Tf-Idf")
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf


corpus_tfidf = create_tfidf(corpus)

Creación del Modelo Espacio-Vector Tf-Idf


En corpus tenemos, para cada project, una lista con sus palabras y el tfidf de cada una. Mostramos las 10 primeras entradas del <primer elemento

In [72]:
corpus_tfidf[0][:10]

[(0, 0.00998019703663961),
 (1, 0.027979188824840854),
 (2, 0.018446982275474397),
 (3, 0.024226462326885523),
 (4, 0.020580016402277544),
 (5, 0.024226462326885523),
 (6, 0.027979188824840854),
 (7, 0.006333751112031634),
 (8, 0.018446982275474397),
 (9, 0.013989594412420427)]

Si queremos saber que palabra es cada uno de estos términos podemos consultar el diccionario

In [36]:
print(dictionary[8],",",dictionary[19],",", dictionary[88])

know , start , multi


## Determinación del número de topicos

Con los elementos que tenemos hasta ahora ya estamos listos para entrenar el modelo LSI/LDA. Pero antes que nada, como dicen los anglosajones "*el elefante en la habitación*", **¿Cuál es el número correcto de tópicos?**

LSA busca identificar un conjunto de tópicos relacionados las descripciones de los proyectos. El número de estos temas N es igual a la dimensión de la matriz de aproximación resultante de la técnica de reducción de dimensión SVD. Este número es un hiperparámetro que se debe ajustar cuidadosamente, es el resultado de la selección de los N valores singulares más grandes de la matriz del corpus tf-idf. 

En principio no hay una respuesta correcta o fácil para la pregunta inicial, depende tanto de los datos que tengamos como de la aplicación que queramos dar a estos datos. Por ejemplo, en ciertos casos puede ser útil tener un número pequeño de tópicos, de mara que se puedan interpretar y "etiquetar".



Vamos a crear una serie de modelos variando el número de tópicos y posteriormente los evaluaremos buscando el número óptimo de tópicos.

In [37]:
trained_models = OrderedDict()
for num_topics in range(10, 201, 10):
    print("Training LDA(k=%d)" % num_topics)
    lda = models.LdaMulticore(
        corpus_tfidf, id2word=dictionary, num_topics=num_topics, workers=4,
        passes=10, iterations=100, random_state=42, 
        eval_every=None, # No evalúa la perplejidad del modelo, lleva demasiado tiempo.
        alpha='asymmetric',  # Demuestra ser mejor que simétrico en la mayoría de los casos
        decay=0.5, offset=64  # Mejores varlores para Hoffman paper
    )
    trained_models[num_topics] = lda

Training LDA(k=10)
Training LDA(k=20)
Training LDA(k=30)
Training LDA(k=40)
Training LDA(k=50)
Training LDA(k=60)
Training LDA(k=70)
Training LDA(k=80)
Training LDA(k=90)
Training LDA(k=100)
Training LDA(k=110)
Training LDA(k=120)
Training LDA(k=130)
Training LDA(k=140)
Training LDA(k=150)
Training LDA(k=160)
Training LDA(k=170)
Training LDA(k=180)
Training LDA(k=190)
Training LDA(k=200)


Podemos mostrar uno de los modelos creados

In [38]:
trained_models[10]

### Evaluación usando la *Coherencia*

Vamos a evaluar cada uno de nuestros modelos de LDA utilizando la coherencia. La **coherencia** es una medida de cuán interpretables son los topicos para los humanos. Se basa en la representación de temas como las N palabras más probables para un topico en particular. Más específicamente, dada la matriz de *topic-term* para LDA, clasificamos cada tema de pesos más altos a más bajos y luego seleccionamos los primeros N términos.

La coherencia esencialmente mide cuán similares son estas palabras entre sí. Existen varios métodos para hacerlo, la mayoría de los cuales han sido explorados en el documento ["Exploring the Space of Topic Coherence Measures"](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf). Los autores realizaron un análisis comparativo de varios métodos, correlacionándolos con juicios humanos. El método llamado coherencia "c_v" resultó ser el más altamente correlacionado. Este y varios de los otros métodos se han implementado en gensim.models.CoherenceModel. Usaremos esto para realizar nuestras evaluaciones.



Recorremos la lista de modelos y obtenemos la **coherencia** de cada uno de ellos, además llevaremos un registro para determinar cual de ellos obtiene el mejor resultado.

In [39]:
best_coherence = 0
best_coherence_key = 0
for key, value in trained_models.items():
    cm = CoherenceModel(model=value, texts=texts_from_readme, dictionary=dictionary, coherence='c_v')
    actual_coherence = cm.get_coherence()
    print("Topic number: {} - Coherence: {}".format(key,actual_coherence))
    if actual_coherence > best_coherence:
        best_coherence = actual_coherence
        best_coherence_key = key
print("\nBest model has {} topics and a coherence of {}".format(best_coherence_key,best_coherence))

Topic number: 10 - Coherence: 0.6100369667801769
Topic number: 20 - Coherence: 0.6454301249689803
Topic number: 30 - Coherence: 0.5732946161572147
Topic number: 40 - Coherence: 0.5819150705285903
Topic number: 50 - Coherence: 0.5640479900317257
Topic number: 60 - Coherence: 0.5834842772221394
Topic number: 70 - Coherence: 0.5864665529488722
Topic number: 80 - Coherence: 0.5843458406779152
Topic number: 90 - Coherence: 0.5808202909323953
Topic number: 100 - Coherence: 0.5864550314662079
Topic number: 110 - Coherence: 0.5864629623402351
Topic number: 120 - Coherence: 0.5840622861067865
Topic number: 130 - Coherence: 0.5822491566269213
Topic number: 140 - Coherence: 0.5821637170808125
Topic number: 150 - Coherence: 0.5807192399865083
Topic number: 160 - Coherence: 0.5791838002268871
Topic number: 170 - Coherence: 0.5823628164466407
Topic number: 180 - Coherence: 0.5802487571615315
Topic number: 190 - Coherence: 0.5758066513214363
Topic number: 200 - Coherence: 0.5789532609364118

Best mod

La prueba nos ha dado como resultado que con **20** tópicos obtenemos los mejores resultados.

Finalmente creamos constantes para los valores seleccionados.
 * **TOTAL_LSA_TOPICS**: Limita el numero de terminos, por supuesto tiene que ver con el tamaño de la muestra, mientras más proyectos tengamos mas terminos tendremos y por tanto la reduccion seria mayor, estamos clusterizando las proyectos en TOTAL_TOPICOS_LSA clusters
 
* **SIMILARITY_THRESHOLD** Umbral de similitud que se debe superar para que dos proyectos se consideren similares (en cualquier caso, posteriormente usaremos los proyectos ordenados por similitud

In [40]:
TOTAL_LSA_TOPICS = best_coherence_key
SIMILARITY_THRESHOLD = 0.4

In [41]:
TOTAL_LSA_TOPICS

20

## Definición del modelo LSA

Para ello usaremos la librería **models** que nos ofrece gensim

In [42]:
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=TOTAL_LSA_TOPICS)

Podemos ver como influyen las palabras en la determinación de los diferentes tópicos, por ejemplo mostramos los 3 primeros tópicos

In [43]:
for i, topic in enumerate(model.print_topics(3)):
    print('Topic {}:'.format(i))
    print(topic[1].replace(' + ', '\n'))
    print('')

Topic 0:
0.131*"file"
0.121*"softwar"
0.121*"python"
0.120*"django"
0.119*"http"
0.117*"com"
0.116*"test"
0.116*"instal"
0.112*"https"
0.104*"set"

Topic 1:
-0.276*"softwar"
0.211*"sublim"
0.169*"diff"
0.166*"file"
0.159*"mxunit"
0.138*"mark"
0.138*"text"
0.136*"folder"
0.135*"temporari"
-0.131*"django"

Topic 2:
-0.309*"softwar"
0.218*"django"
0.175*"transifex"
0.148*"heroku"
-0.142*"kbd"
-0.135*"sublim"
0.126*"app"
-0.120*"jenkinsapi"
0.107*"manag"
-0.105*"replac"



Usando nuestro modelo LSI construimos la matriz de similitud

In [44]:
similarity_matrix = similarities.MatrixSimilarity(model[corpus_tfidf])

Definimos la función auxiliar que dado un documento (correspondiente a la descripción de un proyecto), nos determina la lista de proyectos que superan el umbral de similitud. Para cada proyecto que supere el umbral, almacenaremos el índice dentro de la matriz de proyectos, para localizarla posteriormente, y el grado de similitud. La función nos devolverá la lista ordenada por similitud.

In [45]:
def get_similarities_by_description(doc, model):
    ''' Calcula las similitudes de un documento, expresado este como una lista de palabras'''
    project_similarities = []

    # Convertimos el documento al espacio LSI
    vec_bow = dictionary.doc2bow(doc)
    vec_lsi = model[vec_bow]

    similarities = similarity_matrix[vec_lsi]
    similarities = sorted(enumerate(similarities), key=lambda item: -item[1])

    for sim in similarities:
        similarity_project = int(sim[0])
        similarity_score = sim[1]
        if similarity_score > SIMILARITY_THRESHOLD:
            project_similarities.append((similarity_project, similarity_score))

    return (project_similarities)

A modo de ejemplo, como prueba de concepto, vamos a determinar los proyectos similares a uno dado.

In [46]:
poc_library = ['sys', 'os', 'json', 'codecs', 'shutil','re']
poc_readme_doc = get_words("works on Windows and write down some installation instructions")
project_similarities_by_description = get_similarities_by_description(poc_readme_doc, model)

A continuación, mostramos los 10 proyectos más similares

In [47]:
for similarity in project_similarities_by_description[:10]:
    print("Project: {0}: - Similarity: {1}".format(projects[similarity[0]]["name"], similarity[1]))

Project: illumiprocessor: - Similarity: 0.6922912001609802
Project: tabtabtab-nuke: - Similarity: 0.6359504461288452
Project: heaper: - Similarity: 0.5907448530197144
Project: script.openvpn: - Similarity: 0.5845991373062134
Project: bombolone: - Similarity: 0.5749461650848389
Project: pyrite: - Similarity: 0.5176742672920227
Project: LivelierView: - Similarity: 0.4793180823326111
Project: Website: - Similarity: 0.4740278124809265
Project: python-package-template: - Similarity: 0.4688289761543274
Project: lggr: - Similarity: 0.4656628668308258


El objetivo final no no es encontrar proyectos similares, sino encontrar las librerias que usan esos proyectos similares. Así pues, recorremos esos proyectos e identificamos esas librerías, descartando las que nuestro proyecto ya incluye.

Creamos un diccionario donde la clave es cada libreria y el valor el scoring de esa librería, obtendremos dicho scoring sumando el scoring de similitud de cada proyecto en el que encontremos la librería

Recorremos los 20 proyectos más similares y calculamos el scoring de cada librería usada en esos proyectos

In [48]:
def get_library_similarities_by_description(project_similarities, proyect_libraries):
    library_similarities=defaultdict(float)
    for project in project_similarities[:20]:
        similarity = project[1]
        libraries = projects[project[0]]["library"]
        for library in libraries:
            if library not in proyect_libraries:
                library_similarities[library] += similarity
    return library_similarities

In [49]:
library_similarities_by_description = get_library_similarities_by_description(project_similarities_by_description, poc_library)

Mostramos las 10 librerías más usadas por los proyectos similares

In [50]:
for library in sorted(library_similarities_by_description, key=library_similarities_by_description.get, reverse=True)[:10]:
    print("Library: {0}: - Score: {1}".format(library, library_similarities_by_description[library]))

Library: time: - Score: 4.376614987850189
Library: subprocess: - Score: 3.041726052761078
Library: threading: - Score: 2.844213992357254
Library: datetime: - Score: 2.7599836587905884
Library: hashlib: - Score: 2.7482431530952454
Library: string: - Score: 2.6761107742786407
Library: logging: - Score: 2.559393733739853
Library: distutils.core: - Score: 2.5026374757289886
Library: optparse: - Score: 2.1378364861011505
Library: struct: - Score: 1.946576565504074


# Modelo CountVectorizer

Otro acercamiento que podemos realizar es usar como nivel de similitud las propias librerías que un proyecto usa, es decir buscaremos proyectos que usen las mismas librerías que nosotros ya estamos usando y mostraremos otras librerias que esos mismos proyectos estén usando y nuestro proyecto no.

<span style="color: red; font-family: Arial; font-size: 2em;">TO-DO: Explicar porque no es conveniente usar la técnica anterior con documentos tan cortos</span>

In [51]:
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer

El modelo que construiremos necesita como entrada textos, así que transformamos nuestras listas de palabras en textos, en los que las librerías estarán separados por espacios.

In [52]:
global_texts = [" ".join(project['library']) for project in projects]

Usaremos un objeto de **CountVectorizer**, el cual convierte una colección de documentos de texto en una matriz de conteos de tokens. 

Useremos el parámetro **min_df** con el valor **2**,  denomina comunmente como **corte**, esto hará que al construir el vocabulario, ignorará los términos que tienen una frecuencia de documento más baja que el umbral dado

In [53]:
countVectorizer = CountVectorizer(min_df=1)

Usando el objeto creado, realizamos la transformación de nuestros datos

In [54]:
X_train_data = countVectorizer.fit_transform(global_texts)

A continuación definimos una función que usando los datos anteriores, el objeto countVectorizer y una lista de librerias, calcula la distancia coseno con cada una de las listas de librerias de los proyectos objeto de estudio. La **distancia coseno**  no es propiamente una distancia sino una medida de similaridad entre dos vectores en un espacio que tiene definido un producto interior. 

In [55]:
def get_similarities_by_library(X_train, libraries, countVectorizer):
    def cos_distance(v1, v2):
        return 1 - (v1 * v2.transpose()).sum() / (sp.linalg.norm(v1.toarray()) * sp.linalg.norm(v2.toarray()))
    
    new_doc = " ".join(libraries)
    new_doc_vectorized = countVectorizer.transform([new_doc])[0]
    library_similarities = defaultdict(float)
    for i, doc_vec in enumerate(X_train):
        library_similarities[i] = 1 - cos_distance(doc_vec, new_doc_vectorized)
    return library_similarities

Vamos a realizar una prueba de concepto usando una lista de librerias.

In [56]:
project_similarities_by_library = get_similarities_by_library(X_train_data, poc_library, countVectorizer)

Como hicimos con el análisis por descripciones, podemos mostrar los proyectos ordenados por similitud

In [57]:
for project_id in sorted(project_similarities_by_library, key=project_similarities_by_library.get, reverse=True)[:10]:
     print("Project: {0}: - Similarity: {1}".format(projects[project_id]["name"], project_similarities_by_library[project_id]))

Project: sublime-text-2-mxunit: - Similarity: 0.5773502691896258
Project: Sublime-Text-2-Goto-CSS-Declaration: - Similarity: nan
Project: tabtabtab-nuke: - Similarity: 0.5773502691896258
Project: tingdownload: - Similarity: 0.5270462766947299
Project: python-phabricator: - Similarity: 0.4714045207910318
Project: weather-cli: - Similarity: 0.4714045207910318
Project: python-transifex: - Similarity: 0.4682929057908469
Project: SublimeStringEncode: - Similarity: 0.4629100498862757
Project: DefVectors: - Similarity: 0.45291081365783836
Project: SublimeFileDiffs: - Similarity: 0.40824829046386313


Aunque como ya dijimos entonces, el objetivo es recomendar librerías, no proyectos. Para ello definimos una función que dada la lista anterior, nos muestra la lista de las librerías más usadas.

In [58]:
def get_library_similarities_by_libraries(project_similarities, project_libraries):
    library_similarities=defaultdict(float)
    for project_id in sorted(project_similarities, key=project_similarities.get, reverse=True)[:10]:
        similarity = project_similarities[project_id]
        libraries = projects[project_id]["library"]
        for library in libraries:
            if library not in project_libraries:
                library_similarities[library] += similarity
    return library_similarities

Y la invocamos con nuestra lista de proyectos

In [59]:
library_similarities_by_libraries = get_library_similarities_by_libraries(project_similarities_by_library, poc_library)

Finalmente mostramos el resultado

In [60]:
for library in sorted(library_similarities_by_libraries, key=library_similarities_by_libraries.get, reverse=True)[:10]:
    print("Library: {0}: - Score: {1}".format(library, library_similarities_by_libraries[library]))

Library: sublime: - Score: 1.4485086095397648
Library: sublime_plugin: - Score: 1.4485086095397648
Library: hashlib: - Score: 1.4026074764681544
Library: unittest: - Score: 1.0456431749804729
Library: urllib: - Score: 0.9984507974857617
Library: argparse: - Score: 0.9984507974857617
Library: logging: - Score: 0.9799570903525683
Library: collections: - Score: 0.9428090415820636
Library: time: - Score: 0.9243153344488702
Library: subprocess: - Score: 0.8611591041217015


## Aplicación a un proyecto de ejemplo

Para poner en práctica el sistema de recomendación vamos a usar un proyecto real almacenado en Github. En primer lugar definimos una función que dado un proyecto, lo descargue en local y aplique el mismo procesamiento que hicimos con los proyectos que componen nuestro corpus.

In [61]:
def proccess_url(git_url):
    def process_python_file(project, file_path):
        def add_to_list(item):
            if not item in library:
                library.append(item)

        library = project['library']
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                match = re.search(LIBRARY_PATTERN, line)
                if match:
                    if match.group(1) is not None:
                        add_to_list(match.group(1))
                    else:
                        add_to_list(match.group(2))
        project['library'] = library

    def process_readme_file(project, file_path):
        with open(file_path, 'r') as f:
            project['readme_txt'] = f.read()

    project = dict()
    os.chdir(ROOT_PATH)
    dir_name = uuid.uuid4().hex
    path = ROOT_PATH + "/" + dir_name

    if not os.path.isdir(path):
        os.system(CLONE_COMMAND.format(git_url, dir_name))
        project['git_url'] = git_url
        project['library'] = []
        for root, dirs, files in os.walk(path):
            for file in files:
                try:
                    if file.endswith(".py"):
                        process_python_file(project, os.path.join(root, file))
                    else:
                        if file.lower().startswith("readme."):
                            process_readme_file(project, os.path.join(root, file))
                except:
                    pass

    return project

Y la invocamos con nuestro projecto de ejemplo

In [63]:
sample_project = proccess_url("https://github.com/yazquez/programmable-agents_tensorflow.git")

Podemos ver las librerías que está usando

In [64]:
sample_project['library']

['tensorflow',
 'tensorflow.contrib.layers.python.layers',
 'numpy',
 'math',
 'detector',
 'program',
 'message_passing',
 'gym',
 'ou_noise',
 'critic_network',
 'actor_network',
 'replay_buffer',
 'ou_noise_canonical',
 'critic_network_canonical',
 'actor_network_bn_canonical',
 'replay_buffer_canonical',
 'filter_env',
 'ddpg',
 'gc',
 'filter_env_canonical',
 'ddpg_canonical',
 'numpy.random',
 'collections',
 'random',
 'gym.envs.mujoco']

Así como su descripción (fichero **readme**)

In [65]:
sample_project['readme_txt']

'Programmable Agents\n====================\n\nTensorflow Implementation of Programmable Agents from Google Deepmind.\n\nImplementation is on Tensorflow r1.3.\n\nDDPG script is based on songrotek\'s repo. https://github.com/songrotek/DDPG.git\n\nhttps://arxiv.org/abs/1706.06383\n\n" The agents learn to ground the terms in this language in their environment,\nand can generalize their behavior at test time to execute new programs that refer to\nobjects that were not referenced during training. The agents develop disentangled\ninterpretable representations that allow them to generalize to a wide variety of\nzero-shot semantic tasks." from the paper\n\n![alt tag](https://github.com/jaesik817/programmable-agents_tensorflow/blob/master/figures/pa.PNG)\n\nEnvironment Settings\n-----------------\n\nThis module uses OpenAI GYM. for installing that, please refer https://github.com/openai/gym.\n\nThis module uses Mujoco physics engine and mujoco_py 0.5 branch from OpenAI. For installing that, plea

Si recapitulamos, tenemos las siguientes funciones definidas:

- **get_similarities_by_description**: Devuelve los proyectos similares en funcíon de sus descripciones
- **get_library_similarities_by_description**: Devuelve las librerías comunes usando la lista anterior
- **get_similarities_by_library**: Devuelve los proyectos similares en funcíon de las librerías que se usa
- **get_library_similarities_by_libraries(project_similarities)**: Devuelve las librerías comunes usando la lista anterior


Las aplicamos a nuestro proyecto de ejemplo

Empezamos obteniendo los proyectos más similares al del ejemplo usando las descripciones y a partir de estos proyectos obtenemos las librerías

In [66]:
readme_doc = get_words(sample_project['readme_txt'])
project_similarities_by_description = get_similarities_by_description(poc_readme_doc, model)
library_similarities_by_description = get_library_similarities_by_description(project_similarities_by_description, sample_project['library'])

Mostramos los 10 proyectos más similares

In [67]:
for similarity in project_similarities_by_description[:10]:
    print("Project: {0}: - Similarity: {1}".format(projects[similarity[0]]["name"], similarity[1]))

Project: illumiprocessor: - Similarity: 0.6922912001609802
Project: tabtabtab-nuke: - Similarity: 0.6359504461288452
Project: heaper: - Similarity: 0.5907448530197144
Project: script.openvpn: - Similarity: 0.5845991373062134
Project: bombolone: - Similarity: 0.5749461650848389
Project: pyrite: - Similarity: 0.5176742672920227
Project: LivelierView: - Similarity: 0.4793180823326111
Project: Website: - Similarity: 0.4740278124809265
Project: python-package-template: - Similarity: 0.4688289761543274
Project: lggr: - Similarity: 0.4656628668308258


Hacemos lo mismo usando las librerías del proyecto como medida de similitud

In [68]:
project_similarities_by_library = get_similarities_by_library(X_train_data, poc_library, countVectorizer)
library_similarities_by_libraries = get_library_similarities_by_libraries(project_similarities_by_library, sample_project['library'])

Mostramos los 10 proyectos más similares

In [69]:
for project_id in sorted(project_similarities_by_library, key=project_similarities_by_library.get, reverse=True)[:10]:
     print("Project: {0}: - Similarity: {1}".format(projects[project_id]["name"], project_similarities_by_library[project_id]))

Project: sublime-text-2-mxunit: - Similarity: 0.5773502691896258
Project: Sublime-Text-2-Goto-CSS-Declaration: - Similarity: nan
Project: tabtabtab-nuke: - Similarity: 0.5773502691896258
Project: tingdownload: - Similarity: 0.5270462766947299
Project: python-phabricator: - Similarity: 0.4714045207910318
Project: weather-cli: - Similarity: 0.4714045207910318
Project: python-transifex: - Similarity: 0.4682929057908469
Project: SublimeStringEncode: - Similarity: 0.4629100498862757
Project: DefVectors: - Similarity: 0.45291081365783836
Project: SublimeFileDiffs: - Similarity: 0.40824829046386313


Finalmente mostramos el resultado, o sea las librerías recomendadas

**Librerías recomendadas en función de las descripciones**

In [70]:
for library in sorted(library_similarities_by_description, key=library_similarities_by_description.get, reverse=True)[:10]:
    print("Library: {0}: - Score: {1}".format(library, library_similarities_by_description[library]))

Library: os: - Score: 7.96423065662384
Library: sys: - Score: 6.916517972946167
Library: re: - Score: 4.883899122476578
Library: time: - Score: 4.376614987850189
Library: shutil: - Score: 3.100588083267212
Library: subprocess: - Score: 3.041726052761078
Library: threading: - Score: 2.844213992357254
Library: datetime: - Score: 2.7599836587905884
Library: hashlib: - Score: 2.7482431530952454
Library: string: - Score: 2.6761107742786407


**Librerías recomendadas en función de las librerías comunes**

In [71]:
for library in sorted(library_similarities_by_libraries, key=library_similarities_by_libraries.get, reverse=True)[:10]:
    print("Library: {0}: - Score: {1}".format(library, library_similarities_by_libraries[library]))

Library: sys: - Score: 4.008669625991006
Library: re: - Score: 3.839567647265243
Library: os: - Score: 3.4826033457775614
Library: json: - Score: 2.9784085431435416
Library: codecs: - Score: 1.8564982866072783
Library: sublime_plugin: - Score: 1.4485086095397648
Library: sublime: - Score: 1.4485086095397648
Library: hashlib: - Score: 1.4026074764681544
Library: unittest: - Score: 1.0456431749804729
Library: urllib: - Score: 0.9984507974857617


* https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/lda_training_tips.ipynb
* https://radimrehurek.com/gensim/models/coherencemodel.html
* https://rare-technologies.com/what-is-topic-coherence/
* https://radimrehurek.com/gensim/models/coherencemodel.html

